# FLOTVIS:
## a YOLOv4/TF-Keras based floating plastic debris detector

This code is designed to run on Google Colab <https://colab.research.google.com/>

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/nschang/FLOTVIS"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


# Define Variables

In [ ]:
# uncomment if using Colab
# # mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# '''
# DRIVE_PATH = '/content/drive/MyDrive'
# '''
# select tensorflow version
# %tensorflow_version 1.x

In [19]:
# check current path
!pwd
# ------------------------------------
import os
# ------------------------------------
# format: YYMMDDHHSS
TODAY = '2109022300' # change this to current date
# ------------------------------------
#LAST  = ''
SAVE_PATH = 'FLOTV'
LOG_PATH = 'train'
RESULT_PATH = 'results'
DRIVE_PATH = '/content/drive/MyDrive'
# ------------------------------------
%cd /content/$SAVE_PATH
!pwd

/Volumes/Extreme-SSD/thesis/FLOTVIS


# Clone from Github

In [ ]:
%cd /content/
!git clone https://github.com/nschang/FLOTVIS.git

In [ ]:
# import dataset
# also removes old dataset if present
#!rm -rf results train val VOCdevkit/VOC2007/Annotations VOCdevkit/VOC2007/JPEGImages vision_for_anchors.jpg predict.jpg prediction
LOCAL_PATH = cwd + '/VOCdevkit/VOC2007'
REMOTE_PATH = 'https://dataset.zip'

!wget -nc $LOCAL_PATH/dataset.zip $REMOTE_PATH
%cd $cwd/VOCdevkit/VOC2007/
!unzip dataset.zip
!rm -rf __MACOSX dataset.zip

# Environment Set-up

In [ ]:
# install dependencies
%cd $cwd
!pip install -r $cwd/requirements.txt

In [ ]:
# show installed environment
!pip show tensorflow-estimator
!echo '--------'
!pip show keras
!echo '--------'
!pip show h5py
!echo '--------'
# check tensorflow version and active GPU device
import tensorflow as tf
print('tensorflow version is ', tf.__version__)
tf.test.gpu_device_name() # '/device:GPU:0' means active GPU
# get GPU Info
# !nvidia-smi

# --- Predict ---

# Predict

In [22]:
%cd $cwd
# get FPS
!python3 predict.py --mode='fps'

/Volumes/Extreme-SSD/thesis/FLOTVIS
Using TensorFlow backend.
/Users/nick/.pyenv/versions/3.6.9/envs/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/nick/.pyenv/versions/3.6.9/envs/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/nick/.pyenv/versions/3.6.9/envs/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qi

In [24]:
import os
from tqdm import tqdm
image_ids1 = os.listdir('./test/')
image_ids = [f for f in os.listdir('./test/') if f.endswith(".jpg")]
print(tqdm(image_ids))
print(image_ids1)
print(image_ids)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]
['.DS_Store', 'test.mp4', 'test.jpg']
['test.jpg']


In [ ]:
%cd $cwd
# predict single image
#img = input('Input image filename:')
!python3 predict.py --mode='image' --img='test/test.jpg'

In [ ]:
%cd $cwd
# predict all images in folder
!python3 predict.py --mode='batch' --imgdir='./test/'

In [ ]:
%cd $cwd
# predict video
!python3 predict.py --mode='video' --vid='test/test.mp4'

In [ ]:
%cd $cwd
# predict using camera
!python3 predict.py --mode='camera'

# --- Train ---

# Test

In [ ]:
# reload all modules before execution
%load_ext autoreload
%autoreload 2
# test
%cd $cwd/
!python test.py

# Dataset Check

In [ ]:
# simple script to evaluate the dataset integrity and consistency

# create a new folder Allempty, means to use the xml file as 
# a reference for image checking, if the image does not have 
# corresponding xml file, then move it to Allempty folder

%cd $cwd/VOCdevkit/VOC2007
import os, shutil

def checkJpgXml(dir1, dir2, dir3, is_move=True):
    """
    dir1 is the folder where the image is located
    dir2 is the folder where the annotation files are located
    dir3 is created if the image does not have a corresponding xml file, then the image is put into dir3
    is_move is to confirm whether to move or not, otherwise just print
    """
    if not os.path.exists(dir3):
        os.mkdir(dir3)
    cnt = 0
    for file in os.listdir(dir1):
        f_name,f_ext = file.split(".")
        if not os.path.exists(os.path.join(dir2, f_name+".xml")):
            print(f_name)
            if is_move:
                cnt += 1
                shutil.move(os.path.join(dir1,file), os.path.join(dir3, file))
    if cnt > 0:
        print("There are %d files that do not meet the requirements and have been printed."%(cnt))
    else:
        print("All images and corresponding xml files are one-to-one.")

if __name__ == "__main__":
    dir1 = r"JPEGImages"
    dir2 = r"Annotations"
    dir3 = r"Allempty"
    checkJpgXml(dir1, dir2, dir3, False)

In [ ]:
# number of images
!echo 'number of images'; find $cwd/VOCdevkit/VOC2007/JPEGImages -type f | wc -l
# number of labels (in .xml format)
!echo 'number of labels'; find $cwd/VOCdevkit/VOC2007/Annotations -type f | wc -l

In [ ]:
%cd $cwd/VOCdevkit/VOC2007/
!python $cwd/VOCdevkit/VOC2007/voc2yolo4.py

In [ ]:
%cd $cwd/
!python voc_annotation.py

In [ ]:
# k-means
%cd $cwd/
!python $cwd/kmeans_for_anchors.py
!mv $cwd/model_data/yolo_anchors.txt $cwd/model_data/yolo_anchors.txt.bak
!mv $cwd/yolo_anchors.txt $cwd/model_data/yolo_anchors.txt

In [ ]:
# get anchor
%cd $cwd/
#!python /content/yolov4-keras-2/vision_for_anchors.py
import matplotlib.pyplot as plt
import numpy as np

def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s

# 13x13
def yolo_head(feats, anchors, num_classes):
    # 3
    num_anchors = len(anchors)
    # [1, 1, 1, num_anchors, 2]
    anchors_tensor = np.reshape(anchors, [1, 1, 1, num_anchors, 2])  / 32

    # get x,y grid
    # (13,13, 1, 2)
    grid_shape = np.shape(feats)[1:3] # height, width
    print(grid_shape)
    grid_y = np.tile(np.reshape(np.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]),
        [1, grid_shape[1], 1, 1])
    grid_x = np.tile(np.reshape(np.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]),
        [grid_shape[0], 1, 1, 1])
    grid = np.concatenate([grid_x, grid_y],-1)
    print(np.shape(grid))
    # (batch_size,13,13,3,85)
    feats = np.reshape(feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    # adjust predict val to true val
    # box_xy = center of box
    # box_wh = width and height of box
    box_xy = (sigmoid(feats[..., :2]) + grid)
    box_wh = np.exp(feats[..., 2:4]) * anchors_tensor
    box_confidence = sigmoid(feats[..., 4:5])
    box_class_probs = sigmoid(feats[..., 5:])

  
    fig = plt.figure()
    ax = fig.add_subplot(121)
    plt.ylim(-2,15)
    plt.xlim(-2,15)
    plt.scatter(grid_x,grid_y)
    plt.scatter(5,5,c='black')
    plt.gca().invert_yaxis()


    anchor_left = grid_x - anchors_tensor/2 
    anchor_top = grid_y - anchors_tensor/2 
    print(np.shape(anchors_tensor))
    rect1 = plt.Rectangle([anchor_left[0,5,5,0,0],anchor_top[0,5,5,0,1]],anchors_tensor[0,0,0,0,0],anchors_tensor[0,0,0,0,1],color="r",fill=False)
    rect2 = plt.Rectangle([anchor_left[0,5,5,1,0],anchor_top[0,5,5,1,1]],anchors_tensor[0,0,0,1,0],anchors_tensor[0,0,0,1,1],color="r",fill=False)
    rect3 = plt.Rectangle([anchor_left[0,5,5,2,0],anchor_top[0,5,5,2,1]],anchors_tensor[0,0,0,2,0],anchors_tensor[0,0,0,2,1],color="r",fill=False)

    ax.add_patch(rect1)
    ax.add_patch(rect2)
    ax.add_patch(rect3)

    ax = fig.add_subplot(122)
    plt.ylim(-2,15)
    plt.xlim(-2,15)
    plt.scatter(grid_x,grid_y)
    plt.scatter(5,5,c='black')
    plt.scatter(box_xy[0,5,5,:,0],box_xy[0,5,5,:,1],c='r')
    plt.gca().invert_yaxis()

    pre_left = box_xy[...,0] - box_wh[...,0]/2 
    pre_top = box_xy[...,1] - box_wh[...,1]/2 

    rect1 = plt.Rectangle([pre_left[0,5,5,0],pre_top[0,5,5,0]],box_wh[0,5,5,0,0],box_wh[0,5,5,0,1],color="r",fill=False)
    rect2 = plt.Rectangle([pre_left[0,5,5,1],pre_top[0,5,5,1]],box_wh[0,5,5,1,0],box_wh[0,5,5,1,1],color="r",fill=False)
    rect3 = plt.Rectangle([pre_left[0,5,5,2],pre_top[0,5,5,2]],box_wh[0,5,5,2,0],box_wh[0,5,5,2,1],color="r",fill=False)

    ax.add_patch(rect1)
    ax.add_patch(rect2)
    ax.add_patch(rect3)

    plt.show()
    #
feat = np.random.normal(0,0.5,[4,13,13,75])
anchors = [[142, 110],[192, 243],[459, 401]]
yolo_head(feat,anchors,20)


# Train

In [ ]:
# check tensorflow version and active GPU device
import tensorflow as tf
print('tensorflow version is ', tf.__version__)
tf.test.gpu_device_name() # '/device:GPU:0' means active GPU
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))
# allow GPU growth
# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True



In [ ]:
# Load the TensorBoard notebook extension before training
# in case of error, run: pip uninstall tensorboard-plugin-wit
%load_ext tensorboard
import tensorflow as tf
import datetime, os
%tensorboard --logdir train #--host localhost --port 8088

from tensorboard import notebook
notebook.list() # View open TensorBoard instances

# Control TensorBoard display. If no port is provided, the most recently launched TensorBoard is used
#notebook.display(port=8088, height=1000) 

In [ ]:
# train
%cd $cwd
!python train.py

# Evaluate

In [ ]:
%cd $cwd
!python get_dr_txt.py

In [ ]:
!python get_gt_txt.py

In [ ]:
!python get_map.py